In [2]:
from liblibra_core import *
from libra_py import *
import math
import py3Dmol

/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  """Entry point for launching an IPython kernel.
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  """Entry point for launching an IPython kernel.
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std:

In [3]:
params = {"nsteps":100, "nperiods":1, "scale":250.0, "visualize":0, "verbosity":0, "prefix":"_Si8_cov_flag0"}

R, V, A, M, E = QE_methods.read_md_data("au-c60.xml")

"""
C = QE_methods.read_md_data_cell("au-c60.xml")

print C.get(0,0), C.get(1,0), C.get(2,0)
print C.get(3,0), C.get(4,0), C.get(5,0)
print C.get(6,0), C.get(7,0), C.get(8,0)

H = MATRIX3x3()
H.xx = C.get(0,0);  H.xy = C.get(0,1);  H.xz = C.get(0,2);
H.yx = C.get(1,0);  H.yy = C.get(1,1);  H.yz = C.get(1,2);
H.zx = C.get(2,0);  H.zy = C.get(2,1);  H.zz = C.get(2,2);

tH = H.T()
fR = fold_coords(R, tH, "a")
"""

'\nC = QE_methods.read_md_data_cell("au-c60.xml")\n\nprint C.get(0,0), C.get(1,0), C.get(2,0)\nprint C.get(3,0), C.get(4,0), C.get(5,0)\nprint C.get(6,0), C.get(7,0), C.get(8,0)\n\nH = MATRIX3x3()\nH.xx = C.get(0,0);  H.xy = C.get(0,1);  H.xz = C.get(0,2);\nH.yx = C.get(1,0);  H.yy = C.get(1,1);  H.yz = C.get(1,2);\nH.zx = C.get(2,0);  H.zy = C.get(2,1);  H.zz = C.get(2,2);\n\ntH = H.T()\nfR = fold_coords(R, tH, "a")\n'

### Method 1:

Strachan, A. Normal Modes and Frequencies from Covariances in Molecular Dynamics 
    or Monte Carlo Simulation. J. Chem. Phys. 2003, 120, 1-4.
    
In the following 2 calls *res* is a tuple (w, w_inv_cm, U_v,  w2, w2_inv_cm, U_a)

In [4]:
params.update({"cov_flag":0})
res00 = normal_modes.compute_cov( R, V, A, M, E, params)

Another option is to use data centered around the averages (fluctuations)

In [5]:
params.update({"cov_flag":1})
res01 = normal_modes.compute_cov( R, V, A, M, E, params)


### Method 2:
Pereverzev, A.; Sewell, T. D. Obtaining the Hessian from the Force Covariance Matrix:
    Application to Crystalline Explosives PETN and RDX. J. Chem. Phys. 2015, 142, 134110.
    
In the following 2 calls *res* is a tuple (w_a, w_inv_cm, U_a)
Again, without centering:

In [6]:
params.update({"cov_flag":0})
res20 = normal_modes.compute_cov2( R, A, M, E, 300.0, params)

With the centering:

In [7]:
params.update({"cov_flag":1})
res21 = normal_modes.compute_cov2( R, A, M, E, 300.0, params)

Now, we can select which normal modes to visualize:

In [14]:
mode = 50
ampl = 10000

# Method 1
xyz00v = normal_modes.get_xyz(E,R, M, res00[2], mode)  # velocity covariance
xyz00a = normal_modes.get_xyz(E,R, M, res00[5], mode)  # acceleration covariance
xyz01v = normal_modes.get_xyz(E,R, M, res01[2], mode)  # velocity fluctuation covariance
xyz01a = normal_modes.get_xyz(E,R, M, res01[5], mode)  # acceleration fluctuation covariance

# Method 2
xyz20f = normal_modes.get_xyz(E,R, M, res20[2], mode)  # force covariance
xyz21f = normal_modes.get_xyz(E,R, M, res21[2], mode)  # force fluctuation covariance


view = py3Dmol.view(width=1000,height=800, linked=False,viewergrid=(3,2))
view.addModel(xyz00v,'xyz',{'vibrate': {'frames':10,'amplitude':ampl}}, viewer=(0,0))
view.addModel(xyz01v,'xyz',{'vibrate': {'frames':10,'amplitude':ampl}}, viewer=(0,1))

view.addModel(xyz00a,'xyz',{'vibrate': {'frames':10,'amplitude':ampl}}, viewer=(1,0))
view.addModel(xyz01a,'xyz',{'vibrate': {'frames':10,'amplitude':ampl}}, viewer=(1,1))

view.addModel(xyz20f,'xyz',{'vibrate': {'frames':10,'amplitude':ampl}}, viewer=(2,0))
view.addModel(xyz21f,'xyz',{'vibrate': {'frames':10,'amplitude':ampl}}, viewer=(2,1))

view.setBackgroundColor('0xeeeeee')
view.setStyle({'sphere':{}})
view.animate({'loop': 'backAndForth'})
view.zoomTo()
view.show()

You appear to be running in JupyterLab. You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [9]:
print R.get(30,0)

10.5701046079


In [10]:
x = R.col(0)
x.show_matrix()

In [11]:
print E
print xyz00v

['Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'Au', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']
108
Comment
Au  -0.372  19.245  -2.014  0.000  -0.000  -0.000
Au  -0.250  19.048  11.653  -0.000  0.000  0.000
Au  -0.349  9.843  -2.038  -0.000  0.000  -0.000
Au  -0.231  9.600  11.649  -0.000  -0.000  -0.000
Au  5.073  19.266  -2.013  -0.000  -0.000  0.000
Au  5.200  19.034  11.635  -0.000  -0.000  0.000
Au  5.086  9.830  -1.999  -0.000  -0.000  -0.000
Au  5.210  9.613  11.679  0.000  -0.000  -0.000
Au  10

In [13]:
au='''50
Comment
N     0.000005    0.019779   -0.000003   -0.000   0.000  0.050
Au   -0.372      19.245      19.245      -0.000  -0.000  0.000
Au   -0.250      19.048      19.048      -0.000   0.000  0.000
Au   -0.349       9.843       9.843      -0.000  -0.000  0.050
Au   -0.349       9.843      19.843      -0.000  -0.000  0.050
Au  -0.231   9.600   9.600  -0.000  -0.000  0.000
Au   5.073  19.266  19.266  -0.000   0.000  0.000
Au   5.200  19.034  19.034  -0.000  -0.000  0.000
Au   5.086   9.830   9.830  -0.000   0.000  0.000
Au   5.210   9.613   9.613  -0.000   0.000 -0.000
Au  10.536  19.259  19.259  -0.000   0.000 -0.000
Au  10.652  19.044  19.044  -0.000  -0.000  0.000
Au  10.570   9.828   9.828  -0.000   0.000  0.000
Au  10.655   9.600   9.600  -0.000   0.000  0.000
Au   2.404   8.063   8.063  -0.000  -0.000  0.000
Au   2.417  17.516  17.516  -0.000   0.000  0.000
Au   7.864   8.080   8.080  -0.000   0.000  0.000
Au   7.852  17.527  17.527  -0.000   0.000 -0.000
Au  13.303   8.074   8.074  -0.000   0.000  0.000
Au  13.316  17.533  17.533  -0.000   0.000  0.000
Au  -0.311   3.361   3.361  -0.000   0.000 -0.000
Au  -0.328  12.807  12.807  -0.000   0.000 -0.000
Au   5.122   3.360   3.360  -0.000   0.000  0.000
Au   5.122  12.806  12.806  -0.000  -0.000  0.000
Au  10.575   3.370   3.370  -0.000   0.000  0.000
Au  10.589  12.781  12.781  -0.000  -0.000  0.000
Au  -0.275   6.646   6.646  -0.000  -0.000  0.000
Au  -0.308  16.090  16.090  -0.000  -0.000  0.000
Au   5.156   6.656   6.656  -0.000  -0.000  0.000
Au   5.171  16.069  16.069  -0.000   0.000  0.000
Au  10.623   6.649   6.649  -0.000   0.000 -0.000
Au  10.605  16.101  16.101  -0.000   0.000 -0.000
Au   2.436   1.933   1.933  -0.000   0.000 -0.000
Au   2.453  11.353  11.353  -0.000  -0.000  0.000
Au   7.890   1.954   1.954  -0.000   0.000  0.000
Au   7.887  11.350  11.350  -0.000   0.000 -0.000
Au  13.359   1.935   1.935  -0.000   0.000  0.000
Au  13.329  11.352  11.352  -0.000   0.000  0.000
Au   2.377   5.100   5.100  -0.000  -0.000  0.000
Au   2.491   4.889   4.889  -0.000   0.000  0.000
Au   2.372  14.534  14.534  -0.000  -0.000  0.000
Au   2.477  14.349  14.349  -0.000  -0.000  0.000
Au   7.852   5.101   5.101  -0.000  -0.000  0.000
Au   7.916   4.891   4.891  -0.000   0.000 -0.000
Au   7.807  14.542  14.542  -0.000  -0.000  0.000
Au   7.933  14.341  14.341  -0.000  -0.000  0.000
Au  13.279   5.103   5.103  -0.000   0.000  0.000
Au  13.369   4.905   4.905  -0.000   0.000 -0.000
Au  13.265  14.554  14.554  -0.000   0.000 -0.000
Au  13.384  14.317  14.317  -0.000   0.000  0.000'''

view1 = py3Dmol.view(width=800,height=400)
view1.addModel(au,'xyz',{'vibrate': {'frames':10,'amplitude':500}})

view1.setBackgroundColor('0xeeeeee')
view1.setStyle({'sphere':{}})
view1.animate({'loop': 'backAndForth'})
view1.zoomTo()
view1.show()

You appear to be running in JupyterLab. You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol